In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import random

# SCRAPING TRIPADVISOR
## Installation des dépendances
- !pip install selenium
- !pip install selenium-stealth
- pip install fake-useragent

<img src="scraping.png" width="800" height="200" />

In [2]:
def configurer_driver():
    """
    Configure et retourne un driver Selenium avec options de furtivité
    pour éviter la détection en tant que bot
    """
    options = Options()

    # Désactivation des flags automation
    # options.add_experimental_option("excludeSwitches", ["enable-automation"])
    # options.add_experimental_option('useAutomationExtension', False)
    
    # Désactivation des fonctionnalités de détection d'automatisation
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-blink-features")
    options.add_argument("--no-sandbox")  # Mode sans sandbox
    options.add_argument("--disable-dev-shm-usage")  # Évite les problèmes de mémoire
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")  # Désactivation des extensions
    
    
    # User-agent aléatoire
    # ua = UserAgent()
    # user_agent = ua.random
    # options.add_argument(f'--user-agent={user_agent}')

    # Autres paramètres
    # options.add_argument("--window-size=1920,1080")
    options.add_argument("--start-maximized") # Fenêtre maximisée
    
    driver = webdriver.Chrome(options=options)

    # Masquer WebDriver
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    # Configuration de la furtivité pour ressembler à un navigateur humain
    stealth(driver,
            languages=["fr-FR", "fr"],  # Langues françaises
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL",
            fix_hairline=True,  # Correction des lignes fines
            run_on_insecure_origins=True,
    )

    # Timeouts réalistes
    driver.set_page_load_timeout(30)
    driver.set_script_timeout(20)
    
    return driver

In [3]:
def comportement_humain(driver):
    """
    Simule un comportement de navigation humain
    """
    # Scroll aléatoire
    scroll_height = random.randint(200, 600)
    driver.execute_script(f"window.scrollTo(0, {scroll_height});")
    
    # Pause humaine
    time.sleep(random.uniform(1.0, 3.0))
    
    # Mouvement de souris aléatoire
    # actions = webdriver.ActionChains(driver)
    # actions.move_by_offset(random.randint(10, 100), random.randint(10, 100))
    # actions.perform()

In [4]:
def extraire_avis(driver, url_offre):
    """
    Extrait tous les avis disponibles pour une offre donnée
    
    Args:
        driver: Instance du driver Selenium
        url_offre: URL de l'offre Tripadvisor
    
    Returns:
        Liste des avis collectés
    """
    print(f"Début extraction: {url_offre}")
    
    avis_collectes = []
    page_actuelle = 0
    url_page_courante = url_offre
    pages_sans_nouveaux_avis = 0
    pages_max = 4
    
    try:
        # Parcours des pages d'avis jusqu'à 3 pages sans nouveaux avis
        while pages_sans_nouveaux_avis < 3 and page_actuelle < pages_max:
            page_actuelle += 1
            
            # Chargement avec timeout court
            driver.set_page_load_timeout(25)
            driver.get(url_page_courante)
            
            # Comportement humain aléatoire
            temps_attente = random.uniform(3, 7)
            time.sleep(temps_attente)

            # Scroll humain
            comportement_humain(driver)
            
            # Analyse du contenu HTML
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            avis_page = analyser_avis(soup, url_offre)
            
            # Filtrage des doublons et ajout des nouveaux avis
            avis_avant = len(avis_collectes)
            for avis in avis_page:
                texte_avis = avis.get('texte_avis', '')
                if texte_avis and not any(a.get('texte_avis') == texte_avis for a in avis_collectes):
                    avis_collectes.append(avis)
            
            nouveaux_avis = len(avis_collectes) - avis_avant
            
            print(f"   Page {page_actuelle}: {nouveaux_avis} nouveaux avis ({len(avis_collectes)} total)")
            
            # Vérification si nouveaux avis
            if nouveaux_avis == 0:
                pages_sans_nouveaux_avis += 1
                print(f"   Aucun nouvel avis ({pages_sans_nouveaux_avis}/2)")
            else:
                pages_sans_nouveaux_avis = 0

            # Vérification si on a atteint la limite de pages
            if page_actuelle >= pages_max:
                print(f"   Limite de {pages_max} pages atteinte")
                break
            
            # Recherche page suivante avec pause
            url_page_suivante = trouver_page_suivante(soup, url_page_courante)
            
            if not url_page_suivante or url_page_suivante == url_page_courante:
                print(f"   Plus de pages disponibles")
                break
                
            url_page_courante = url_page_suivante

            # Pause stratégique entre pages
            if page_actuelle % 3 == 0:
                time.sleep(random.uniform(8, 15))  # Longue pause occasionnelle
            else:
                time.sleep(random.uniform(2, 5))   # Courte pause normale
            
        print(f"Extraction terminée: {len(avis_collectes)} avis collectés sur {page_actuelle} pages")
        
    except Exception as e:
        print(f"Erreur lors de l'extraction: {str(e)}")
        print(f"Avis récupérés avant l'erreur: {len(avis_collectes)}")
    
    return avis_collectes

In [5]:
def analyser_avis(soup, url_offre):
    """
    Analyse le HTML et extrait les données des avis
    
    Args:
        soup: Objet BeautifulSoup de la page
        url_offre: URL de l'offre pour référence
    
    Returns:
        Liste des avis parsés
    """
    avis = []
    
    # Recherche de tous les conteneurs d'avis
    conteneurs_avis = soup.find_all('div', attrs={'data-automation': 'reviewCard'})
    
    for conteneur in conteneurs_avis:
        try:
            donnees_avis = extraire_details_avis(conteneur, url_offre)
            if donnees_avis and donnees_avis.get('texte_avis'):
                avis.append(donnees_avis)
        except Exception as e:
            print(f"   Erreur sur un avis: {str(e)}")
            continue
    
    return avis

In [6]:
def extraire_details_avis(conteneur, url_offre):
    """
    Extrait les détails spécifiques d'un avis depuis son conteneur HTML
    
    Args:
        conteneur: Élément HTML contenant un avis
        url_offre: URL de l'offre pour référence
    
    Returns:
        Dictionnaire avec tous les détails de l'avis
    """
    avis = {
        'url_offre': url_offre,
        'titre_offre': '',
        'titre_avis': '',
        'texte_avis': '',
        'note_avis': '',
        'date_avis': '',
        'auteur_avis': '',
        'localisation_auteur': '',
        'contributions_auteur': '',
        'type_voyage': '',
        'votes_utiles': '0'
    }
    
    try:
        # Titre de l'avis
        element_titre = conteneur.find('span', class_='yCeTE')
        if element_titre:
            avis['titre_avis'] = element_titre.get_text(strip=True)
        
        # Texte de l'avis
        element_texte = conteneur.find('div', class_='biGQs _P VImYz AWdfh')
        if element_texte:
            span_texte = element_texte.find('span', class_='yCeTE')
            if span_texte:
                avis['texte_avis'] = span_texte.get_text(strip=True)
        
        # Note de l'avis
        element_note = conteneur.find('svg', class_='evwcZ')
        if element_note:
            titre_note = element_note.find('title')
            if titre_note:
                texte_note = titre_note.get_text(strip=True)
                # Extraction du chiffre de la note (ex: "5 sur 5 bulles" -> 5)
                correspondance = re.search(r'(\d+)\s+sur\s+5', texte_note)
                if correspondance:
                    avis['note_avis'] = correspondance.group(1)
        
        # Date de l'avis
        element_date = conteneur.find('div', class_='biGQs', string=re.compile(r'Écrit le'))
        if element_date:
            avis['date_avis'] = element_date.get_text(strip=True).replace('Écrit le ', '')
        
        # Auteur de l'avis
        element_auteur = conteneur.find('span', class_='biGQs _P ezezH')
        if element_auteur:
            avis['auteur_avis'] = element_auteur.get_text(strip=True)
        
        # Localisation et contributions de l'auteur
        element_localisation = conteneur.find('div', class_='vYLts')
        if element_localisation:
            texte_localisation = element_localisation.get_text(strip=True)
            # Séparation localisation et contributions
            parties = texte_localisation.split('•')
            if len(parties) > 0:
                avis['localisation_auteur'] = parties[0].strip()
            if len(parties) > 1:
                avis['contributions_auteur'] = parties[1].strip()
        
        # Type de voyage
        element_voyage = conteneur.find('div', class_='RpeCd')
        if element_voyage:
            avis['type_voyage'] = element_voyage.get_text(strip=True)
        
        # Votes utiles
        element_votes = conteneur.find('span', class_='kLqdM')
        if element_votes:
            avis['votes_utiles'] = element_votes.get_text(strip=True)
            
    except Exception as e:
        print(f"      Détail manquant dans un avis: {str(e)}")
    
    return avis

In [7]:
def trouver_page_suivante(soup, url_actuelle):
    """
    Trouve le lien vers la page suivante dans la pagination
    
    Args:
        soup: Objet BeautifulSoup de la page actuelle
        url_actuelle: URL de la page courante
    
    Returns:
        URL de la page suivante ou None si non trouvée
    """
    try:
        # Recherche par aria-label "Page suivante"
        lien_suivant = soup.find('a', attrs={'aria-label': 'Page suivante'})
        if lien_suivant and lien_suivant.get('href'):
            return urljoin("https://www.tripadvisor.fr", lien_suivant['href'])
        
        # Recherche alternative dans la pagination
        pagination = soup.find('div', class_='GMYGA')
        if pagination:
            liens = pagination.find_all('a', href=True)
            for lien in liens:
                if 'next' in lien.get('href', '') or 'or' in lien.get('href', ''):
                    return urljoin("https://www.tripadvisor.fr", lien['href'])
                    
    except Exception as e:
        print(f"   Erreur recherche page suivante: {str(e)}")
    
    return None

In [ ]:
def extraire_titre_offre(soup):
    """
    Extrait le titre de l'offre depuis la page
    
    Args:
        soup: Objet BeautifulSoup de la page
    
    Returns:
        Titre de l'offre ou chaîne vide si non trouvé
    """
    try:
        # Plusieurs sélecteurs possibles pour le titre de l'offre
        selecteurs_titre = [
            'h1[data-automation="mainH1"]',
            'h1.HjBfq',
            '.kUaMB',
            'h1'
        ]
        
        for selecteur in selecteurs_titre:
            element_titre = soup.select_one(selecteur)
            if element_titre:
                titre = element_titre.get_text(strip=True)
                if titre and len(titre) > 0:
                    return titre
    
    except Exception as e:
        print(f"   Erreur extraction titre offre: {str(e)}")
    
    return ""


def obtenir_liens_offres_accueil(driver):
    """
    Récupère les liens des offres depuis la page d'accueil de Tripadvisor
    
    Args:
        driver: Instance du driver Selenium
    
    Returns:
        Liste des URLs des offres trouvées
    """
    print("Recherche des offres sur la page d'accueil...")
    
    driver.get("https://www.tripadvisor.fr/")
    time.sleep(6)  # Pause pour chargement complet
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    liens_offres = []
    
    # Patterns de recherche pour les liens d'offres
    patterns_liens = [
        'a[href*="/AttractionProductReview-"]',
        'a[href*="/ActivityProductReview-"]',
        'a[href*="/TourProductReview-"]'
    ]
    
    for pattern in patterns_liens:
        liens = soup.select(pattern)
        for lien in liens:
            href = lien.get('href')
            if href and not href.startswith('http'):
                url_complete = urljoin("https://www.tripadvisor.fr", href)
                if url_complete not in liens_offres:
                    liens_offres.append(url_complete)
    
    print(f"{len(liens_offres)} offres trouvées")
    return liens_offres #[:15]  # Limite à 15 offres pour les tests


def principal():
    """
    Fonction principale orchestrant l'extraction des avis Tripadvisor
    """
    driver = configurer_driver()
    tous_avis = []
    
    try:
        # Récupération des liens d'offres depuis l'accueil
        liens_offres = obtenir_liens_offres_accueil(driver)
        
        # Extraction des avis pour chaque offre
        for i, url_offre in enumerate(liens_offres, 1):
            print(f"\nTraitement de l'offre {i}/{len(liens_offres)}")
            
            # Extraction du titre de l'offre
            driver.get(url_offre)
            time.sleep(3)
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')
            titre_offre = extraire_titre_offre(soup_offre)
            
            # Extraction des avis avec le titre de l'offre
            avis = extraire_avis(driver, url_offre)
            
            # Ajout du titre de l'offre à chaque avis
            for avi in avis:
                avi['titre_offre'] = titre_offre
            
            tous_avis.extend(avis)
            
            print(f"Total actuel: {len(tous_avis)} avis collectés")
            
            # Pause entre les offres pour éviter le blocage
            if i < len(liens_offres):
                time.sleep(3)
        
        # Sauvegarde des données
        if tous_avis:
            df = pd.DataFrame(tous_avis)
            
            # Nettoyage des données
            df = df.drop_duplicates(subset=['texte_avis'], keep='first')
            df = df[df['texte_avis'].str.len() > 5]  # Suppression des avis trop courts
            
            # Sauvegarde en CSV
            nom_fichier = './DATA/tripadvisor_avis.csv'
            df.to_csv(nom_fichier, index=False, encoding='utf-8-sig', sep=';')
            
            print(f"\nSUCCÈS: {len(df)} avis sauvegardés dans '{nom_fichier}'")
            
            # Aperçu des données
            print("\nAperçu des données:")
            print(f"Colonnes: {df.columns.tolist()}")
            print(f"\nExemple d'avis:")
            for i, ligne in df.head(2).iterrows():
                print(f"  {i+1}. [{ligne['note_avis']}/5] {ligne['texte_avis'][:100]}...")
                
        else:
            print("Aucun avis extrait")
            
    except Exception as e:
        print(f"Erreur générale: {str(e)}")
        
    finally:
        driver.quit()
        print("\nExtraction terminée")


# Démarrage du script
print("Démarrage de l'extraction des avis Tripadvisor...")
principal()

Démarrage de l'extraction des avis Tripadvisor...
Recherche des offres sur la page d'accueil...
20 offres trouvées

Traitement de l'offre 1/20
Début extraction: https://www.tripadvisor.fr/AttractionProductReview-g187147-d15040216-Eiffel_Tower_Guided_Tour_by_Elevator_with_Optional_Summit-Paris_Ile_de_France.html
   Page 1: 10 nouveaux avis (10 total)
   Page 2: 10 nouveaux avis (20 total)
   Page 3: 10 nouveaux avis (30 total)
   Page 4: 10 nouveaux avis (40 total)
   Limite de 4 pages atteinte
Extraction terminée: 40 avis collectés sur 4 pages
Total actuel: 40 avis collectés

Traitement de l'offre 2/20
Début extraction: https://www.tripadvisor.fr/AttractionProductReview-g187895-d14917017-Leonardo_Da_Vinci_Museum_Entrance_Ticket-Florence_Tuscany.html
   Page 1: 10 nouveaux avis (10 total)
   Page 2: 10 nouveaux avis (20 total)
   Page 3: 10 nouveaux avis (30 total)
   Page 4: 10 nouveaux avis (40 total)
   Limite de 4 pages atteinte
Extraction terminée: 40 avis collectés sur 4 pages
Tot